In [2]:
import org.apache.spark.mllib.regression.{LabeledPoint,LinearRegressionWithSGD}
import org.apache.spark.sql.SparkSession
import org.apache.spark.mllib.linalg.{Vector,Vectors}

In [29]:
// The code was removed by Watson Studio for sharing.

credentials = Map(apiKey -> 1OcYKgYhwRYrcTmooNxtQnmT5f2P5AhDBVJmZnQJ5TbX, serviceId -> iam-ServiceId-7d4019a9-639e-45a5-beca-c0c18f60292a, endPoint -> https://s3-api.us-geo.objectstorage.service.networklayer.com, iamServiceEndpoint -> https://iam.ng.bluemix.net/oidc/token)
configurationName = os_30ac23a0f916475aac293915c17c261c_configs
cos = com.ibm.ibmos2spark.CloudObjectStorage@30d7a96d


lastException: Throwable = null


com.ibm.ibmos2spark.CloudObjectStorage@30d7a96d

In [30]:
val dfData1 = spark.
    read.format("org.apache.spark.sql.execution.datasources.csv.CSVFileFormat").
    option("header", "true").
    option("inferSchema", "true").
    load(cos.url("sparkmldatatypes-donotdelete-pr-avzxrdgnnkhrc4", "housing.csv"))
dfData1.show(5)

+------------------------------------------------------------------------------------------------+
| 0.00632  18.00   2.310  0  0.5380  6.5750  65.20  4.0900   1  296.0  15.30 396.90   4.98  24.00|
+------------------------------------------------------------------------------------------------+
|                                                                             0.02731   0.00  ...|
|                                                                             0.02729   0.00  ...|
|                                                                             0.03237   0.00  ...|
|                                                                             0.06905   0.00  ...|
|                                                                             0.02985   0.00  ...|
+------------------------------------------------------------------------------------------------+
only showing top 5 rows



dfData1 = [ 0.00632  18.00   2.310  0  0.5380  6.5750  65.20  4.0900   1  296.0  15.30 396.90   4.98  24.00: string]


[ 0.00632  18.00   2.310  0  0.5380  6.5750  65.20  4.0900   1  296.0  15.30 396.90   4.98  24.00: string]

In [31]:
dfData1.printSchema()

root
 |--  0.00632  18.00   2.310  0  0.5380  6.5750  65.20  4.0900   1  296.0  15.30 396.90   4.98  24.00: string (nullable = true)



In [22]:
val regressionDataSet = dfData1.map{ r =>
    val columns = r.getAs[String](0).split(' ').filter(!_.isEmpty)
    LabeledPoint(columns(13).toDouble, Vectors.dense(
    columns(0).toDouble,
    columns(1).toDouble,
    columns(2).toDouble,
    columns(3).toDouble,
    columns(4).toDouble,
    columns(5).toDouble,
    columns(6).toDouble,
    columns(7).toDouble        
    ))
}.rdd.cache()

//regressionDataSet.collect().foreach(println(_))

regressionDataSet = MapPartitionsRDD[60] at rdd at <console>:52


MapPartitionsRDD[60] at rdd at <console>:52

In [27]:
val numIterations = 1000
val stepsSGD = .001

numIterations = 1000
stepsSGD = 0.001


0.001

In [24]:
val myModel = LinearRegressionWithSGD.train(regressionDataSet,numIterations,stepsSGD)

myModel = org.apache.spark.mllib.regression.LinearRegressionModel: intercept = 0.0, numFeatures = 8


org.apache.spark.mllib.regression.LinearRegressionModel: intercept = 0.0, numFeatures = 8

In [28]:
val predictedLabelValue = regressionDataSet.map { lp =>
    val predictedValue = myModel.predict(lp.features)
    (lp.label, predictedValue)
}


Name: org.apache.spark.SparkException
Message: Task not serializable
StackTrace:   at org.apache.spark.util.ClosureCleaner$.ensureSerializable(ClosureCleaner.scala:345)
  at org.apache.spark.util.ClosureCleaner$.org$apache$spark$util$ClosureCleaner$$clean(ClosureCleaner.scala:335)
  at org.apache.spark.util.ClosureCleaner$.clean(ClosureCleaner.scala:159)
  at org.apache.spark.SparkContext.clean(SparkContext.scala:2292)
  at org.apache.spark.rdd.RDD$$anonfun$map$1.apply(RDD.scala:371)
  at org.apache.spark.rdd.RDD$$anonfun$map$1.apply(RDD.scala:370)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
  at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
  at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
  at org.apache.spark.rdd.RDD.map(RDD.scala:370)
  ... 42 elided
Caused by: java.io.NotSerializableException: com.ibm.ibmos2spark.CloudObjectStorage
Serialization stack:
	- object not serializable (class: com.ibm.ibmos2spa

In [ ]:
println("Intercept set:",myModel.intercept)
println("Model Weights:",myModel.weights)

In [ ]:
predictedLabelValue.takeSample(false,5).foreach(println(_))

In [ ]:
val MSE = predictedLabelValue.map{ case(l, p) => math.pow((l - p),
   2)}.reduce(_ + _) / predictedLabelValue.count

In [ ]:
val RMSE = math.sqrt(MSE)
println("training Mean Squared Error = " + MSE)
println("training Root Mean Squared Error = " + RMSE)
